[View in Colaboratory](https://colab.research.google.com/github/kimitakem/colab_ex/blob/master/sample.ipynb)

# Tensorflow Eager executiomモードのチュートリアル要約
## ライブラリのインポート

In [1]:
import tensorflow as tf
tf.enable_eager_execution()




Check Eager Mode: True


## Tensorの演算
- 足し算：`tf.add`



In [5]:
print("Tensorflow Version: {}".format(tf.__version__))
print("Check Eager Mode: {}".format(tf.executing_eagerly()))




Tensorflow Version: 1.8.0
Check Eager Mode: True


ヘディングです

ヘディング2です